In [1]:
import numpy as np

In [7]:
rgen = np.random.default_rng(12)

In [8]:
rgen.integers(low=0, high=1_000_00, size=10)

array([61269, 25082, 97298, 94675,  6441, 18932, 19948, 17929, 58191,
       34988])

In [9]:
rgen = np.random.default_rng(12)
rgen.integers(low=0, high=1_000_00, size=10)

array([61269, 25082, 97298, 94675,  6441, 18932, 19948, 17929, 58191,
       34988])

In [10]:
rgen = np.random.default_rng(None)
rgen.integers(0, 1_000_000, size=10)

array([685587, 327191, 378517, 943350, 838964, 429287, 461126, 408292,
       447598,   9549])

In [11]:
from sklearn.linear_model import LogisticRegression
mod = LogisticRegression()

In [13]:
hasattr(mod, 'random_state')

True

In [15]:
rgen.choice(list('abc'), size=10, replace=True)

array(['b', 'b', 'c', 'a', 'b', 'b', 'b', 'a', 'a', 'a'], dtype='<U1')